# Rule Based Relation-Extraction

In [ ]:
import os
import pandas as pd
import re
import spacy 
from spacy import displacy

# Load the en_core_web_sm model
nlp = spacy.load("en_core_web_lg")

```
# https://stackoverflow.com/a/47784683
from dataclasses import dataclass

@dataclass(eq=True)
class Relation:
    subj: str
    rel: str
    obj: str
    type: str
Relation('ben','is','here','DIRECT')
```

## Dataset

In [ ]:
texts =  [
    # https://www.bbc.co.uk/news/world-us-canada-60177979
    (
        "The US East Coast is hunkering down as a major blizzard hits the region for the first time in four years."
        "The storm is forecast to stretch from the Carolinas to Maine, packing hurricane-force winds in coastal parts."
        "Five states have declared emergencies."
        "Mayor Michelle Wu of Boston, a city that is no stranger to snowfall, said the storm could be 'historic'."
        "More than two feet of snow could fall in New England."
        "Weather officials also warn of flooding near the coast."
        "Over 5,000 US flights were cancelled between Friday and Sunday, according to FlightAware."
        "Forecasters say there is a chance the storm, known as a Nor\'easter, will blanket the Boston area with up to 2ft (61cm) of snow."
        ),
    # https://www.bbc.co.uk/news/business-60163814
    (  
        "Apple sales soared in the key Christmas shopping season, despite constraints due to a global shortage of microchips."
        "Sales at the iPhone giant rose 11% to a record $123.9bn (£92.6bn) in the October to December period, beating forecasts."
        "Shares jumped more than 4% in after-hours trade, as the report suggested the firm's pandemic boom is continuing."
        "Apple has seen purchases skyrocket during the pandemic as people spend more time online."
        "The firm's market value briefly hit the $3tn milestone in early January though its share price has slipped more recently amid weeks of market turmoil."
        ),
    # https://news.sky.com/story/staycation-frenzy-spurs-center-parcs-owner-to-prepare-4bn-sale-12527982
    (
        "Sky News has learnt that Brookfield Property Partners, the Canadian property giant, is paving the way to sell Center Parcs UK potentially as soon as this year."
        "City sources said this weekend that Brookfield had engaged the accountancy firm PriceWaterhouseCoopers to assist with preparations for a sale process."
        "Investment banks have yet to be formally appointed to handle an auction, and one person close to the process said it was possible that Brookfield would decide to retain the business for a longer period if it did not secure a sufficiently attractive offer."
        "Center Parcs is one of the most famous brands in the British leisure industry, drawing millions of visitors annually to its five UK sites and the latest addition to its portfolio, at Longford Forest in Ireland."
    )
]

##  Part-Of-Speech (POS) tags

In linguistics and grammar, A part of speech or part-of-speech (POS) is the category of a word that have similar grammatical properties. 
For instance "nouns" are words for real things like people, places and objects. Words that describe nouns are called "adjectives" such as: tall, smart, large. 

Applications in Natural Language Processing (NLP) apply linguistic rules and machine learning models to predict and assign which POS tags apply by evaluating word position and context. Popular NLP packages such as NLTK and spaCy include this functionality OOTB. To read more about POS, see this [POS summary](https://towardsdatascience.com/part-of-speech-tagging-for-beginners-3a0754b2ebba), the spaCy [documentation](https://spacy.io/usage/linguistic-features#pos-tagging) and [SO explanation](https://stackoverflow.com/questions/40288323/what-do-spacys-part-of-speech-and-dependency-tags-mean), and this [POS tag reference list](https://sites.google.com/site/partofspeechhelp/#TOC-Welcome).

We can build a basic relation-extraction process by using grammar patterns / part of speech patterns to identify related nouns within a text. A simple rule might be:

```
Proper Noun - Verb - Proper Noun
```

Using spaCy, we can now iterate over each sentence and identify where this POS pattern occurs.

In [ ]:
nouns = ['NNP','NN','NNS']
verbs = ["VBZ","VB","VBG"]
relations = list()

for text in texts:
    doc = nlp(text)
    for e,sent in enumerate(doc.sents):
        chain = list()
        for a in sent:
            if a.tag_ in nouns: # find first NOUND
                chain.append(a)
                for b in sent[a.i:]: # find ROOT, alternatively VERBS
                    if (b.dep_ == 'ROOT') and len(chain) == 1:
                        chain.append(b)
                        for c in sent[b.i:]: # find second NOUN
                            if c.tag_ in nouns and len(chain) == 2: 
                                chain.append(c)
                                
                                # reset chain and print result
                                relations.append(chain)
                                pos_chain = ' '.join([f"{i} ({i.tag_}|{i.dep_})" for i in sent[a.i:c.i+1]])
                                print(chain,'\n',pos_chain,'\n')
                                chain = list()

So, this simple approach seems OK at finding sentences that contain related entities.

- US --- **hunkering** ---> Blizzard
- Sales --- **soared** ---> Christmas
- Sky --- **learnt** ---> Brookfield

## Relation Extraction

The problem with the above approach is that it relies on an extensive list of *Part-Of-Speech* tag patterns. This won't scale for most problems as nouns and verbs come in a wide variety of forms and with modifiers etc. For instance, you generally want to capture compound term phrases and patterns such as:
```
Metro-North worker = NNP-HYPH-NNP
Killed by = VBZ-IN
```

To improve our method we can:
 1. Better capture and relfect things and objects - collectively named "entities".
     
 
 1. Develop POS patterns and rules to identify and extract relations between two or more entities. 

 1. Train a probabilistic model to identify relation triplets such as [Stanford, OLLIE - see reddit]
 
### Capture entities

A spacy pipeline with components for Named Entity Recognition (NER) and Noun Chunks is used to capture entities. Here, we could define some sensible rules or limit the number and type of entities to control what information will be represented in our knowledge graph.

In [ ]:
try:
    nlp.add_pipe(nlp.create_pipe("merge_entities"))
    nlp.add_pipe(nlp.create_pipe("merge_noun_chunks"))
    print(nlp.pipe_names)
except:
    print(nlp.pipe_names)


doc = nlp(texts[2])
print('\n', ' '.join([f"{d} ({d.tag_}|{d.dep_})" for d in doc]),'\n')
spacy.displacy.render(doc, style='ent')

print('Entities:')
for t in doc:
    if t.ent_type_ != '': print('\t',t,t.ent_type_)

print('Noun chunks:')
for chunk in doc.noun_chunks:
    print('\t',chunk.text, )
    #chunk.root.text, chunk.root.dep_,chunk.root.head.text)

### Capture relations

For each sentence, relations can be extracted by iterating through the entity pairs and noun chunks, and yielding the ROOT VERB and other VERB terms. Spacy's dependency parser operates on each sentence in isolation and so it is not possible to extract relations across sentences with this approach.

In [ ]:
def get_entities(span,ent_types=None,noun_chunks=False):
    entities = dict()
    if ent_types is None:
        ent_types = ['ORG','PERSON','GPE','NORP','LOC','PRODUCT']
        
    if noun_chunks:
        entities.update({int(f"{i.start}{i.end}"): {"span":i, "type":"NOUN_CHUNK"} for i in span.noun_chunks})
        
    entities.update({int(f"{i.start}{i.end}"): {"span":i, "type":i.label_} for i in span.ents if i.label_ in ent_types})
    keys = sorted(entities)
    return entities, keys

In [ ]:
for sent in doc.sents: 
    s_term_pos = " ".join([f"{t} ({t.tag_}|{t.dep_})" for t in sent])
    entities, keys = get_entities(sent)

    if len(keys) > 1:
        pairs = [(x,y) for x,y in zip(keys,keys[1:])]
        print('\n',s_term_pos)
        # print(keys,pairs)
        for p in pairs:
            start,end = entities[p[0]], entities[p[1]]
            for w in doc[start['span'].start:end['span'].end]:
                if w.tag_ in ['VBZ','VBN','VBG','VBD','VB']:
                    print(f"\t>>>\t",f"{start['span']} - {w}({w.tag_}|{w.dep_}) - {end['span']}")

In [ ]:
# spacy.displacy.render(doc, style='ent')
# spacy.displacy.render(sent, style='dep',)
# spacy.explain('attr')

Better - but not great. There's a few additional rules we could add to expand and improve relationship capture. 

- capture direct ascendants and decedents https://spacy.io/usage/linguistic-features#navigating-around

In [ ]:
relations = list()

In [ ]:
def get_direct_relations(sent,entities):
    ent_text = [v['span'].text for v in entities.values()]
    relations = list()

    for v in entities.values():
        
        if v['span'].n_lefts > 0:
            lf = [i for i in v['span'].lefts if i.text in ent_text]
            if len(lf) > 0: 
                relations.append((lf[0],'=IS',v,'DIRECT'))
        if v['span'].n_rights > 0:
            rt = [i for i in v['span'].rights if i.text in ent_text]
            if len(rt) > 0: 
                relations.append((v['span'].text,'=IS',rt[0],'DIRECT'))

    if len(relations) > 0:
        return relations
    else:
        return None


for sent in list(doc.sents)[:]: 
    s_term_pos = " ".join([f"{t} ({t.tag_}|{t.dep_})" for t in sent])
    entities, keys = get_entities(sent, noun_chunks=True)
    
    sentence_relations = get_direct_relations(sent,entities)
    if sentence_relations is not None:
        relations+=sentence_relations
        
        print(s_term_pos)
        for r in sentence_relations:
            print('\t',r)

- get relationships between the subject and object term dependencies.

In [ ]:
def get_ent_pos_relations(doc, sentence_entities):
    """
    Entity to POS relations
    """
    SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
    OBJECTS = ["pobj","dobj", "dative", "attr", "oprd"]
    SUB_OBJ = SUBJECTS+OBJECTS
    
    feature = [  "ROOT",
                 "aux","relcl",
                 "acomp", "advcl", "advmod", "amod", "appos", "nn", "nmod", "ccomp", "complm",
                 "hmod", "infmod", "xcomp", "rcmod", "poss"," possessive",
                 "compound",
                 "prep"]

    relations = list()
    for e in sentence_entities.values():
        prev = list()
        for t in doc[e['span'].end:sent.end]:
            if t.dep_ in ['pobj','nsubj','dojb']: 
                terms = doc[e['span'].end:t.i]
                rels = [t for t in terms if t.dep_ in feature if t.idx not in prev]
                prev.extend([x.idx for x in rels]) # previous terms
                
                if len(rels) > 0:
                    rels = ' '.join([x.text for x in rels])
                    relations.append((e['span'].text,rels,t.text,'ENT_POS'))
    return relations

In [ ]:
for sent in list(doc.sents): 
    s_term_pos = " ".join([f"{t} ({t.tag_}|{t.dep_})" for t in sent])
    entities, keys = get_entities(sent, noun_chunks=True)
    
    sentence_relations = get_ent_pos_relations(doc,entities)
    if sentence_relations is not None:
        relations+=sentence_relations

        print(s_term_pos)
        for r in sentence_relations:
            print('\t',r)

In [ ]:
pd.DataFrame(relations)

- We could also consider [sentence simplification](https://stackoverflow.com/questions/65227103/clause-extraction-long-sentence-segmentation-in-python)

## Notes
spaCy uses the terms **head** and **child** to describe the words connected by a single arc in the dependency tree. The term **dep** is used for the arc label, which describes the type of syntactic relation that connects the child to the head.
 
```
 for token in doc:
    print(token,[token.head],list(token.children),end='\r')
```

## References
 - [information-extraction-with-dominating-rules](https://github.com/philipperemy/information-extraction-with-dominating-rules)
 - [Pruning Knowledge Graphs](http://philipperemy.github.io/information-extract/)
 - [OLLIE](https://www.reddit.com/r/LanguageTechnology/comments/bovsf5/we_release_opiec_the_largest_open_information/)
 - [Clausie](https://github.com/mmxgn/clausiepy)
 - [Minie](https://github.com/mmxgn/miniepy/graphs/contributors)
 - [Knowledge Graph – A Powerful Data Science Technique to Mine Information from Text](https://www.analyticsvidhya.com/blog/2019/10/how-to-build-knowledge-graph-text-using-spacy/)
 - [Spacy subject-object extraction](https://github.com/NSchrading/intro-spacy-nlp/blob/master/subject_object_extraction.py)
     -  [SO dependency parsing](https://stackoverflow.com/questions/39763091/how-to-extract-subjects-in-a-sentence-and-their-respective-dependent-phrases)
 - [sentence simplification](https://stackoverflow.com/questions/65227103/clause-extraction-long-sentence-segmentation-in-python)